### First we need to install MedCAT

In [ ]:
! pip install medcat==0.3.3.1
# Get the scispacy model
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.7 MB/s 
     |████████████████████████████████| 5.7 MB 11.1 MB/s 
     |████████████████████████████████| 753.4 MB 6.7 kB/s 
     |████████████████████████████████| 24.2 MB 81.2 MB/s 
     |████████████████████████████████| 4.0 MB 41.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

**Restart the runtime if on colab, sometimes necessary after installing models**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.utils.vocab import Vocab
from medcat.prepare_cdb import PrepareCDB

Installing the missing models for scispacy

Installing the missing models for scispacy



In [ ]:
!mkdir -p data
DATA_DIR = "./data/"

In [ ]:
!wget https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/cdb_simple.csv -P ./data/
!wget https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/cdb_advanced.csv -P ./data/
!wget https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/vocab_data.txt -P ./data/

--2022-05-26 04:40:20--  https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/cdb_simple.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-26 04:40:20 ERROR 404: Not Found.

--2022-05-26 04:40:20--  https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/cdb_advanced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-26 04:40:20 ERROR 404: Not Found.

--2022-05-26 04:40:20--  https://raw.githubusercontent.com/CogStack/MedCAT/master/tutorial/data/vocab_data.txt
Resolving raw.githubu

# Building a Vocabulary

The first of the two required models when running MedCAT is a Vocabulary model (Vocab). The model is used for two things: (1) Spell checking; and (2) Word Embedding. 

The Vocab is very simple and you can easily build it from a file that is structured as below:
```
<token>\t<word_count>\t<vector_embedding_separated_by_spaces>
```
`token` - Usually a word or subword if you are using Byte Pair Encoding or something similar.

`word_count` - The count for this word in your dataset or in any large dataset (wikipedia also works nicely).

`vector_embedding_separated_by_spaces` - precalculated vector embedding, can be from Word2Vec or BERT or Whatever

---
An example with 3-dim embedding would be:
```
house	34444	 0.3232 0.123213 1.231231
dog	14444	0.76762 0.76767 1.45454
.
.
.
```
The file is basically a TSV, but should not have any heading. 

---

**NOTE**: If spelling is important for your use-case, take care that there are no misspelled words in the Vocab.

In [ ]:
from medcat.utils.vocab import Vocab
vocab = Vocab()
vocab.add_words(path = "")
# Save the new Vocab
vocab.save_dict(DATA_DIR)

FileNotFoundError: ignored

In [ ]:
# Let's have a look at an example, I've created a small vocabulary with only 2 words (the ones from above)
#and saved them into a text file. Let's try to create a vocabulary from this two words.
file_path = DATA_DIR + "vocab_data.txt"

vocab = Vocab()
vocab.add_words(path=file_path)

FileNotFoundError: ignored

**And that is everything, with this we have built our vocab and no futher training is necessary.**

---

A couple of useful functions of the vocab are presented below

In [ ]:
# To see the words in the vocab
vocab.vocab.keys()

dict_keys(['house', 'dog'])

In [ ]:
# If you want to add words manually (one by one) use:
vocab.add_word("test", cnt=31, vec=[1.42, 1.44, 1.55], replace=True)
vocab.vocab.keys()

dict_keys(['house', 'dog', 'test'])

In [ ]:
# To get a vector of word use:
vocab.vec("house")

array([0.3232  , 0.123213, 1.231231])

In [ ]:
# Or to get the count
vocab['house']

34444

In [ ]:
# To chec is a word in the vocab:
"house" in vocab

True

### Before we save the vocab model, we need to create the unigram table for negative sampling

In [ ]:
# This is necessary after each change of the vocabulary (when we add new words)
vocab.make_unigram_table()

### Save the model

In [ ]:
vocab.save_dict(DATA_DIR + "vocab.dat")

### Load the model

In [ ]:
vocab = Vocab()
vocab.load_dict(DATA_DIR + "vocab.dat")

# Building the Concept Database (CDB)

The second model we are going to need when using MedCAT is the ConceptDB (CDB). This database holds a list of all concepts that we would like to detect and link to. For a lot of medical use-cases we would use giant databases like the UMLS or SNOMED. But, MedCAT can be used with any database no matter how big/small it is. 

To prepare the CDB we start off with a CSV with the following structure:
```
cui,str
1,kidney failure
7,CoVid 2
7,coronavirus
```
This is the most basic version of the CSV file, it has only:

`cui` - The concept unique identifier, this is simply an `ID` in your database

`str` - String/Name of that concept. It is important to write all possible names and abbreviations for a concept of interest.

If you have once concept that has multiple different names (like the one above with cui=7), you can simply add multiple rows with the same concept ID and MedCAT will merge that during the build phase.

## The Full CSV Specification
```
cui,str,onto,tty,tui,sty,desc,is_unique,examples
1,Kidney Failure|failure of kidneys|KF,SNOMED,PM,T047,Disease,Description of the concept,The patient was diagnosed with kidney failure
.
.
.
```
This fileds are optional, anyone can be included or left out in your CSV.

`onto` - Source ontology, e.g. HPO, SNOMED, HPC,...

`tty` - Term type e.g. PN - Primary Name. Primary names are important and I would always recommend to add this fields when creating your CDB. 

`tui` - Semantic type identifier - e.g. T047 (taken from UMLS). A list of all semantic types can be found [here](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt). 

`sty` - Semantic type - e.g. Disease

`desc` - Description of this concept

`examples` - Examples of this concept in a sentence (use short examples, not whole documents).


***Note***: If one concept has multiple names, you can also separate the different names by a "|" - pipe - symbol 

In [ ]:
# Let's try building the concept databse from a simple CSV
prep_cdb = PrepareCDB(vocab=vocab)

# Crete an array for paths to CSV files that will be used to build our CDB
paths = [DATA_DIR + 'cdb_simple.csv']
cdb = prep_cdb.prepare_csvs(paths)

Done: 0


In [ ]:
import site
from importlib import reload
reload(site)

<module 'site' from '/usr/lib/python3.6/site.py'>

In [ ]:
print(cdb.cui2original_names)

{'1': {'kidney failure'}, '7': {'coronavirus', 'CoVid 2'}}


In [ ]:
prep_cdb = PrepareCDB(vocab=vocab)

# To build fromm the advanced CSV example
paths = [DATA_DIR + 'cdb_advanced.csv']
cdb = prep_cdb.prepare_csvs(paths)

Done: 0


**That is all, nothing else is necessary to build the CDB**

---

Some useful functions of the cdb are below

In [ ]:
# To display all cuis and names in the db - note that MedCAT merged the names
print(cdb.cui2original_names)

{'1': {'kidney failure'}, '7': {'coronavirus', 'CoVid 2'}}


In [ ]:
# We have a link from tui to cui
print(cdb.tui2cuis)

{'T047': {'1'}}


In [ ]:
# Or vice versa - from cui to tui
print(cdb.cui2tui)

{'1': 'T047'}


In [ ]:
# Description is also there
print(cdb.cui2desc)

{'1': 'Description of the concept\n\nnan'}


### Save the model

In [ ]:
cdb.save_dict(DATA_DIR + "cdb.dat")

### Load the model

In [ ]:
cdb = CDB()
cdb.load_dict(DATA_DIR + "cdb.dat")

# End

This is everything you need to create your own MedCAT models. In the next notebook you will see how to train and use these models to annotate documents. 